본 파일은 알바몬 페이지를 크롤링 하기 위함임..
https://www.albamon.com/list/gi/mon_area_list.asp?page=2&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=

이게 링크이며 이를 토대로 크롤링 진행!

In [44]:


from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import re
import requests

session = requests.Session()

title = [] #알바title = [] #알바 제목

result = pd.DataFrame()  # 결과저장용 DataFrame
pagenum = 1


while(pagenum < 2):  #열개 페이지에 대하여 크롤링
    url = 'https://www.albamon.com/list/gi/mon_area_list.asp?page=1&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=' #이건 알바천국
    url = url.replace('argu', str(pagenum))  # 페이지 1에대한 경우 이후 이는 변수로 처리할 것.

#subcontent > form > div.gListWrap > table > tbody
    pagenum = pagenum + 1 #다음페이지 준비
    #html = urllib.request.urlopen(url)
    page = session.get(url)
    c = page.content
    soup = BeautifulSoup(c, 'html.parser')
    #print(soup)
    title_ = soup.select_one('#subcontent > form > div.gListWrap > table > tbody')
    view = title_.get_text()
    view_list = np.array(view.split())#
    print(view_list)
    view_list = np.delete(view_list, np.where(view_list == '스크랩' | view_list == '새창   view_list = np.array(view.split())#
    print(view_list)
    view_list = np.delete(view_list, np.where(view_list == '스크랩' | view_list == '새창'))


    


['스크랩' '시흥시' '전체' '㈜엔잡/' '장기근속자' '지원/280만' '삼성빌트인'
 '가전(주5일/주간/대기업)남,여사원대모집' '미리보기' '새창' '3,500,000원' '08:30~17:30' '21분전'
 '스크랩' '시흥시' '전체' '라이또PC시흥정왕동점' '-라이또PC방에서' '월,화,금' '아르바이트생을' '모집합니다'
 '미리보기' '새창' '8,720원' '16:00~23:00' '21분전' '스크랩' '시흥시' '정왕동' '케이투이노베이션'
 '[사무직/주5일/8시간/여자경리]정왕역아웃소싱' '사무직' '미리보기' '새창' '1,850,000원' '10:00~19:00'
 '21분전' '스크랩' '시흥시' '정왕동' '㈜IBK서비스' '미화직원' '정규직' '신규채용(IBK기업은행' '시화공단)'
 '미리보기' '새창' '867,691원' '시간협의' '22분전' '스크랩' '시흥시' '전체' '라이또PC시흥정왕동점'
 '-라이또PC방' '주말(금,토)야간' '아르바이트생' '모집-' '미리보기' '새창' '8,720원' '23:00~09:00'
 '23분전' '스크랩' '시흥시' '금이동' '한길창호' '[월280만]' '유리공장' '&' '창호공장' '/' '생산직'
 '채용합니다' '미리보기' '새창' '2,800,000원' '08:30~18:00' '24분전' '스크랩' '시흥시' '배곧동'
 '쿠팡풀필먼트서비스' '유한회사' '[시흥쿠팡]익일지급#기간자유#출퇴근셔틀#동반환영#초' '미리보기' '새창' '91,000원'
 '18:00~03:00' '27분전' '스크랩' '시흥시' '정왕동' '쿠팡풀필먼트서비스' '유한회사'
 '[시흥쿠팡]#꿀알바#주휴#단기#장기#출퇴근셔틀/식사제' '미리보기' '새창' '87,200원' '18:00~03:00'
 '28분전' '스크랩' '시흥시' '정왕동' '쿠팡풀필먼트서비스' '유한회사'
 '[시흥쿠팡]익일지급#기간자유#출퇴근셔틀#동반환영#초' '미리보기' '새창' '91,000원' '18

In [50]:
pagenum = 1
while(pagenum < 2):  #열개 페이지에 대하여 크롤링
    url = 'https://www.albamon.com/list/gi/mon_area_list.asp?page=1&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=' #이건 알바천국
    url = url.replace('argu', str(pagenum))  # 페이지 1에대한 경우 이후 이는 변수로 처리할 것.

#subcontent > form > div.gListWrap > table > tbody
    pagenum = pagenum + 1 #다음페이지 준비
    #html = urllib.request.urlopen(url)
    page = session.get(url)
    c = page.content
    soup = BeautifulSoup(c, 'html.parser')
    #print(soup)
    title_ = soup.find_all('td')
    for conte in title_

[<td align="center" width="240">
<div><iframe frameborder="0" height="35" marginheight="0" marginwidth="0" scrolling="no" src="//yellow.contentsfeed.com/RealMedia/ads/adstream_sx.ads/albamon/sub@Frame1" title="광고" width="195"></iframe></div>
</td>, <td align="center" valign="top">
<div><iframe frameborder="0" height="110" marginheight="0" marginwidth="0" scrolling="no" src="//yellow.contentsfeed.com/RealMedia/ads/adstream_sx.ads/albamon/sub@Top" title="광고" width="495"></iframe></div>
</td>, <td align="center" width="240">
<div><iframe frameborder="0" height="35" marginheight="0" marginwidth="0" scrolling="no" src="//yellow.contentsfeed.com/RealMedia/ads/adstream_sx.ads/albamon/sub@Frame2" title="광고" width="195"></iframe></div>
</td>, <td class="area">
<div>
<span class="scrap" id="dev_scrap_btn__75589951">
<button onclick="getTogether_gi('75589951', '_75589951');" title="스크랩" type="button">스크랩</button>
</span>시흥시 전체<div class="scrapTip" id="dev_scrap_div_75589951"></div>
</div>
</td>, 